In [1]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
data = spark.read.parquet("/FileStore/tables/VenmoSample_snappy-e020d.parquet")
data.cache()   # cache to speed up following operations
display(data)

## Q7
[1 pt]: First, create your dependent variable Y, i.e. the total number of transactions at
lifetime point 12. In other words, for every user, you need to count how many transactions she/he
had committed during her/his twelve months in Venmo.

In [4]:
data.createOrReplaceTempView("datatable")

In [5]:
user_tran = sqlContext.sql(
  '''
  WITH union_user AS(
  SELECT user, datetime, MIN(datetime) OVER (PARTITION BY user) AS first_day
  FROM (
	SELECT user1 AS user, datetime FROM datatable
	UNION
	SELECT user2 AS user, datetime FROM datatable)t)

  SELECT user, COUNT(*) AS user_tran_1Y
  FROM union_user
  WHERE datetime BETWEEN first_day AND ADD_MONTHS(first_day, 12)
  GROUP BY user
  ORDER BY user
  
  '''
)
display(user_tran)

## Q8 
[2 pts]: Create the recency and frequency variables. In CRM, this predictive framework is
known as RFM. Here, you don’t have monetary amounts, so we will focus on just RF. Recency
refers to the last time a user was active, and frequency is how often a user uses Venmo in a
month. You need to compute these metrics across a user’s lifetime in Venmo (from 0 up to 12).
For example, if a user has used Venmo twice during her first month in Venmo with the second
time being on day x, then her recency in month 1 is “30-x” and her frequency is 2/30.

In [7]:
user_RF = sqlContext.sql(
  '''
 WITH union_user AS(
  SELECT user, datetime, MIN(datetime) OVER (PARTITION BY user) AS first_day
  FROM (
	SELECT user1 AS user, datetime FROM datatable
	UNION
	SELECT user2 AS user, datetime FROM datatable)t),

  user_period AS(
    SELECT user, datetime, first_day, 
          CASE WHEN datetime = first_day THEN 0
               WHEN DATEDIFF(datetime, first_day) <= 30 THEN 1
               WHEN DATEDIFF(datetime, first_day) <= 60 THEN 2 
               WHEN DATEDIFF(datetime, first_day) <= 90 THEN 3
               WHEN DATEDIFF(datetime, first_day) <= 120 THEN 4
               WHEN DATEDIFF(datetime, first_day) <= 150 THEN 5
               WHEN DATEDIFF(datetime, first_day) <= 180 THEN 6
               WHEN DATEDIFF(datetime, first_day) <= 210 THEN 7
               WHEN DATEDIFF(datetime, first_day) <= 240 THEN 8
               WHEN DATEDIFF(datetime, first_day) <= 270 THEN 9
               WHEN DATEDIFF(datetime, first_day) <= 300 THEN 10
               WHEN DATEDIFF(datetime, first_day) <= 330 THEN 11
               WHEN DATEDIFF(datetime, first_day) <= 360 THEN 12 END AS period
    FROM union_user
    WHERE DATEDIFF(datetime, first_day) <= 360),
    
    tran_recency AS (
      SELECT user, datetime, first_day, period,
            CASE WHEN period = 0 THEN 0
                 WHEN period = 1 THEN DATEDIFF(DATE_ADD(first_day, 30),datetime)
                 WHEN period = 2 THEN DATEDIFF(DATE_ADD(first_day, 60),datetime)
                 WHEN period = 3 THEN DATEDIFF(DATE_ADD(first_day, 90),datetime)
                 WHEN period = 4 THEN DATEDIFF(DATE_ADD(first_day, 120),datetime)
                 WHEN period = 5 THEN DATEDIFF(DATE_ADD(first_day, 150),datetime)
                 WHEN period = 6 THEN DATEDIFF(DATE_ADD(first_day, 180),datetime)
                 WHEN period = 7 THEN DATEDIFF(DATE_ADD(first_day, 210),datetime)
                 WHEN period = 8 THEN DATEDIFF(DATE_ADD(first_day, 240),datetime)
                 WHEN period = 9 THEN DATEDIFF(DATE_ADD(first_day, 270),datetime)
                 WHEN period = 10 THEN DATEDIFF(DATE_ADD(first_day, 300),datetime)
                 WHEN period = 11 THEN DATEDIFF(DATE_ADD(first_day, 330),datetime)
                 WHEN period = 12 THEN DATEDIFF(DATE_ADD(first_day, 360),datetime) END AS tran_recency
      FROM user_period)

  SELECT DISTINCT user, period, 
        MIN(tran_recency) OVER (PARTITION BY user, period) AS recency,
        CASE WHEN period = 0 THEN 1
            ELSE  COUNT(*) OVER (PARTITION BY user, period) / 30 END AS frequency
  FROM tran_recency
  '''
)
display(user_RF)

In [8]:
user_RF.count()

In [9]:
user_tran.count()

##Q9 
[2 pts]: For each user’s lifetime point, regress recency and frequency on Y. Plot the MSE
for each lifetime point. In other words, your x-axis will be lifetime in months (0-12), and your yaxis
will be the MSE. (Hint: Don’t forget to split your data into train and test sets).

In [11]:
user_all = user_tran.join(user_RF, on=['user'], how='inner')
display(user_all)

In [12]:
user_all.count()

Out[39]: 8112886

In [13]:
# save to storage in case cluster stopped running
user_all.write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/user_all-2.csv")
# display(user_all)

In [14]:
# read from storage
user_all = spark.read.format('csv').options(header='true', inferSchema='true')\
    .load('dbfs:/FileStore/user_all-2.csv')
user_all.createOrReplaceTempView("user_all")

In [15]:
display(user_all)

user,user_tran_1Y,period,recency,frequency
46,2,0,0,1.0
46,2,12,5,0.03333333333333333
275,2,0,0,1.0
275,2,1,7,0.03333333333333333
2504,2,0,0,1.0
2504,2,12,27,0.03333333333333333
2813,3,0,0,1.0
2813,3,9,9,0.03333333333333333
2813,3,10,11,0.03333333333333333
2949,1,0,0,1.0


In [16]:
mse_vec1 = []
for i in range(13):
    subset = user_all.filter(user_all['period']==i)
    
    #select independent variables
    assembler = VectorAssembler(inputCols = ['recency','frequency'], outputCol ='features')
    output = assembler.transform(subset)
    
    #model data
    finalData = output.select('features','user_tran_1Y')
    
    #regress
    trainData, testData = finalData.randomSplit([0.7, 0.3])
    lrModel = LinearRegression(labelCol ='user_tran_1Y', featuresCol='features')
    lrEstimator = lrModel.fit(trainData)
    testResults = lrEstimator.evaluate(testData)
    mse = testResults.rootMeanSquaredError
    mse_vec1.append(mse)
    
plt.scatter(list(range(13)),mse_vec1)
plt.plot(list(range(13)), mse_vec1)
plt.title('Y on Recency and Frequency MSE plot')
plt.line(np.mean(mse_vec1),0,12)
plt.text(1,np.mean(mse_vec1),np.mean(mse_vec1))
display(plt.show())

In [17]:
plt.scatter(list(range(13)),mse_vec1)
plt.plot(list(range(13)), mse_vec1)
plt.title('Y on Recency and Frequency MSE plot')
plt.hlines(np.mean(mse_vec1),0,12)
plt.text(1,np.mean(mse_vec1)+0.1,np.mean(mse_vec1))
display(plt.show())

In [18]:
np.mean(mse_vec1)

Out[43]: 4.86612323201878

In [19]:
# display the feature coeffcient in the model 
pd.DataFrame(['recency','frequency'],lrEstimator.coefficients)

,0
0.002311,recency
79.882380,frequency


### Test on Single Period

In [21]:
mse_vec_test = []

subset = user_all.filter(user_all['period']==0)

#select independent variables
assembler = VectorAssembler(inputCols = ['recency','frequency'], outputCol ='features')
output = assembler.transform(subset)

#model data
finalData = output.select('features','user_tran_1Y')

#regress
trainData, testData = finalData.randomSplit([0.7, 0.3])
lrModel = LinearRegression(labelCol ='user_tran_1Y', featuresCol='features')
lrEstimator = lrModel.fit(trainData)
testResults = lrEstimator.evaluate(testData)
mse = testResults.rootMeanSquaredError
mse_vec_test.append(mse)
mse_vec_test

In [22]:
subset1 = user_all.filter(user_all['period']==1)

#select independent variables
assembler = VectorAssembler(inputCols = ['recency','frequency'], outputCol ='features')
output = assembler.transform(subset)

#model data
finalData = output.select('features','user_tran_1Y')

#regress
trainData, testData = finalData.randomSplit([0.7, 0.3])
lrModel = LinearRegression(labelCol ='user_tran_1Y', featuresCol='features')
lrEstimator = lrModel.fit(trainData)
testResults = lrEstimator.evaluate(testData)
mse = testResults.rootMeanSquaredError
mse_vec_test.append(mse)
mse_vec_test

##Q10 Spending Profile and Recency Frequency
[5 pts]: For each user’s lifetime point, regress recency, frequency AND her spending
behavior profile on Y. Plot the MSE for each lifetime point like above. Did you get any
improvement?

In [24]:
# read user profile from storage
spending_profile = spark.read.csv("/FileStore/tables/part_1_category_percentage_by_lifetime.csv", header="true")
spending_profile.cache()   # cache to speed up following operations
display(spending_profile)

user,life_point,count_activity,count_food,count_people,count_event,count_travel,count_transportation,count_utility,count_cash,count_illegal
2866,0,null,null,null,null,null,null,null,null,null
3918,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3918,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4935,0,null,null,null,null,null,null,null,null,null
5300,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5300,6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5300,0,null,null,null,null,null,null,null,null,null
6620,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
20735,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28170,0,null,null,null,null,null,null,null,null,null


In [25]:
# user profile join user_tran
user_all_spending_profile = user_all.join(spending_profile, (user_all.user == spending_profile.user) & (user_all.period == spending_profile.life_point), how='inner').drop(spending_profile.user)
display(user_all_spending_profile)

In [26]:
# save to storage in case cluster stopped runing
user_all_spending_profile.write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/user_all_spending_profile.csv")
# display(user_all_spending_profile)

In [27]:
# read from storage
user_all_spending_profile = spark.read.format('csv').options(header='true', inferSchema='true')\
    .load('dbfs:/FileStore/user_all_spending_profile.csv')
user_all_spending_profile.createOrReplaceTempView("user_all_spending_profile")

In [28]:
#show the columns and select the ones for regression 
user_all_spending_profile=user_all_spending_profile.fillna(0)

In [29]:
#show the table 
display(user_all_spending_profile)

user,user_tran_1Y,period,recency,frequency,life_point,count_activity,count_food,count_people,count_event,count_travel,count_transportation,count_utility,count_cash,count_illegal
863,3,8,2,0.06666666666666667,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1508,3,0,0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1666,3,8,11,0.03333333333333333,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2324,1,0,0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2351,1,0,0,1.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2526,2,0,0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2719,8,0,0,1.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2719,8,4,23,0.03333333333333333,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2772,2,3,9,0.03333333333333333,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3205,1,0,0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Regress Y on Recency Frequncy variable and user spending profile
mse_vec2 = []
for i in range(13):
    subset = user_all_spending_profile.filter(user_all_spending_profile['period']==i)
    
    #select independent variables
    assembler = VectorAssembler(inputCols =['recency','frequency','count_activity','count_food','count_people','count_event','count_travel','count_transportation','count_utility','count_cash','count_illegal'],outputCol ='features')
    output = assembler.transform(subset)
    
    #model data
    finalData = output.select('features','user_tran_1Y')
    
    #regress
    trainData, testData = finalData.randomSplit([0.7, 0.3])
    lrModel_2 = LinearRegression(labelCol ='user_tran_1Y', featuresCol='features')
    lrEstimator_2 = lrModel_2.fit(trainData)
    testResults_2 = lrEstimator_2.evaluate(testData)
    mse = testResults_2.rootMeanSquaredError
    mse_vec2.append(mse)

In [31]:
#plot the MSE 
plt.scatter(list(range(13)),mse_vec2)
plt.plot(list(range(13)), mse_vec2)
plt.title('Y with Recency,Frequency and user Spending Profile MSE plot')
plt.hlines(np.mean(mse_vec2),0,12)
plt.text(1,np.mean(mse_vec2)+0.1,np.mean(mse_vec2))
display(plt.show())

In [32]:
# display the feature coeffcient in the model in the last model
pd.DataFrame(['recency','frequency','count_activity','count_food','count_people','count_event','count_travel','count_transportation','count_utility','count_cash','count_illegal'],lrEstimator_2.coefficients)

,0
0.000233,recency
79.513349,frequency
-0.505466,count_activity
-0.123251,count_food
0.072359,count_people
-0.613393,count_event
-0.520554,count_travel
0.197769,count_transportation
0.114751,count_utility
-0.001326,count_cash


##Q10 -Social Network Metrics
[5 pts] : For each user’s lifetime point, regress her social network metrics on Y. Plot the MSE for each lifetime point like above. What do you observe? How do social network metrics compare with the RF framework? What are the most informative predictors?

In [34]:
# read user profile from storage
social_matrics = spark.read.csv("FileStore/tables/social_matrics-dc0d8.csv", header="true")
social_matrics.cache()   # cache to speed up following operations
display(social_matrics)

user,month,friend,friend_of_friend,coeff,pagerank
2,0,1,0,0,0.5531666462011291
2,1,1,0,0,0.5531666462011291
2,2,1,0,0,0.5531666462011291
2,3,1,0,0,0.5531666462011291
2,4,1,0,0,0.5531666462011291
2,5,1,0,0,0.5531666462011291
2,6,1,0,0,0.5531666462011291
2,7,1,0,0,0.5531666462011291
2,8,1,0,0,0.5531666462011291
2,9,1,0,0,0.5531666462011291


In [35]:
social_matrics.count()

In [36]:
# user social matrics join user_tran
user_all_social_matrics = user_all.join(social_matrics, (user_all.user == social_matrics.user) & (user_all.period == social_matrics.month), how='inner').drop(social_matrics.user)
display(user_all_social_matrics)

user,user_tran_1Y,period,recency,frequency,month,friend,friend_of_friend,coeff,pagerank
43,18,7,6,0.03333333333333333,7,19,48,0,1.9987567579915
946,8,0,0,1.0,0,2,2,0,1.799699813336965
1060,2,0,0,1.0,0,2,1,0,1.3462294727355013
1079,6,3,27,0.03333333333333333,3,3,8,0,1.795299629535947
1655,7,1,12,0.03333333333333333,1,3,6,0,1.638134128758356
2391,8,11,2,0.03333333333333333,11,9,31,0,0.4079666137499459
2492,5,2,9,0.03333333333333333,2,4,0,0,1.274906300120366
2847,7,12,15,0.03333333333333333,12,5,15,0,1.963047140962693
3352,3,9,6,0.03333333333333333,9,4,15,0,2.5647289413563445
5269,1,0,0,1.0,0,1,1,0,0.4679993086252958


In [37]:
#check the records after join
user_all_social_matrics.count()

Out[34]: 8112886

In [38]:
# save to storage in case cluster stopped runing
user_all_social_matrics.write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/user_all_social_matrics.csv")
# display(user_all_spending_profile)

In [39]:
# read from storage
user_all_social_matrics = spark.read.format('csv').options(header='true', inferSchema='true')\
    .load('dbfs:/FileStore/user_all_social_matrics.csv')
user_all_social_matrics.createOrReplaceTempView("user_all_social_matrics")

In [40]:
#show the table 
display(user_all_social_matrics)

user,user_tran_1Y,period,recency,frequency,month,friend,friend_of_friend,coeff,pagerank
863,3,8,2,0.06666666666666667,8,3,17,0,0.7417858705101853
1508,3,0,0,1.0,0,2,3,0,1.301258844533854
1666,3,8,11,0.03333333333333333,8,3,7,0,1.3064697156888003
2324,1,0,0,1.0,0,1,6,0,0.7818825780141897
2351,1,0,0,1.0,0,2,3,0,1.9918457145836768
2526,2,0,0,1.0,0,2,6,0,0.6484689750862116
2719,8,0,0,1.0,0,6,1,0,3.74559275250642
2719,8,4,23,0.03333333333333333,4,7,9,0,3.74559275250642
2772,2,3,9,0.03333333333333333,3,2,9,0,1.0452561615539795
3205,1,0,0,1.0,0,1,3,0,0.5917951279647015


In [41]:
# Regress Y on Recency Frequncy variable and user spending profile
mse_vec3 = []
lr_coef=[]
for i in range(13):
    subset = user_all_social_matrics.filter(user_all_social_matrics['period']==i)
    
    #select independent variables
    assembler = VectorAssembler(inputCols =['friend','friend_of_friend','coeff','pagerank'],outputCol ='features')
    output = assembler.transform(subset)
    
    #model data
    finalData = output.select('features','user_tran_1Y')
    
    #regress
    trainData, testData = finalData.randomSplit([0.7, 0.3])
    lrModel_3 = LinearRegression(labelCol ='user_tran_1Y', featuresCol='features')
    lrEstimator_3 = lrModel_3.fit(trainData)
    lr_coef.append(lrEstimator_3.coefficients)
    testResults_3 = lrEstimator_3.evaluate(testData)
    mse = testResults_3.rootMeanSquaredError
    mse_vec3.append(mse)
    

In [42]:
#Plot the MSE
plt.scatter(list(range(13)),mse_vec3)
plt.plot(list(range(13)), mse_vec3)
plt.title('Y on Social Network Metrics MSE plot')
plt.hlines(np.mean(mse_vec3),0,13)
plt.text(1,np.mean(mse_vec3)+0.01,np.mean(mse_vec3))
display(plt.show())

In [43]:
#display the coefficient for models 
lr_coef

Out[33]: [DenseVector([1.0573, 0.0029, 0.8057, 1.4516]),
 DenseVector([1.2298, 0.0081, 1.5838, 1.2013]),
 DenseVector([1.0446, 0.0094, 1.9154, 1.2375]),
 DenseVector([1.168, 0.0045, 1.9247, 0.8387]),
 DenseVector([1.0127, 0.0057, 2.0978, 0.9726]),
 DenseVector([1.0997, 0.0027, 2.312, 0.6311]),
 DenseVector([1.0704, 0.0017, 2.2359, 0.5973]),
 DenseVector([0.925, 0.0027, 2.2809, 0.6852]),
 DenseVector([0.9944, 0.0007, 2.2828, 0.4413]),
 DenseVector([0.8248, 0.0012, 2.2249, 0.5963]),
 DenseVector([0.806, 0.0013, 2.1574, 0.5746]),
 DenseVector([0.7418, 0.0012, 2.2129, 0.5187]),
 DenseVector([0.704, 0.0013, 1.9213, 0.4809])]

### Test on single period

In [45]:
mse_vec_test = []

subset = user_all_social_matrics.filter(user_all_social_matrics['period']==0)

#select independent variables
assembler = VectorAssembler(inputCols = ['friend','friend_of_friend','coeff','pagerank'], outputCol ='features')
output = assembler.transform(subset)

#model data
finalData = output.select('features','user_tran_1Y')

#regress
trainData, testData = finalData.randomSplit([0.7, 0.3])
lrModel = LinearRegression(labelCol ='user_tran_1Y', featuresCol='features')
lrEstimator = lrModel.fit(trainData)
testResults = lrEstimator.evaluate(testData)
mse = testResults.rootMeanSquaredError
mse_vec_test.append(mse)


In [46]:
#test on the first month data
subset1 = user_all_social_matrics.filter(user_all_social_matrics['period']==1)

#select independent variables
assembler = VectorAssembler(inputCols = ['friend','friend_of_friend','coeff','pagerank'], outputCol ='features')
output = assembler.transform(subset)

#model data
finalData = output.select('features','user_tran_1Y')

#regress
trainData, testData = finalData.randomSplit([0.7, 0.3])
lrModel = LinearRegression(labelCol ='user_tran_1Y', featuresCol='features')
lrEstimator = lrModel.fit(trainData)
testResults = lrEstimator.evaluate(testData)
mse = testResults.rootMeanSquaredError
mse_vec_test.append(mse)
mse_vec_test

Out[43]: [2.54809299145653, 2.5480466690622117]

## Q11 Social Network Metrics & Social Network Spending Profile

[5 pts] : 
For each user’s lifetime point, regress her social network metrics and the
spending behavior of her social network on Y. Plot the MSE for each lifetime point like
above. Does the spending behavior of her social network add any predictive benefit
compared to Q10?

In [48]:
# user social matrics join user_tran
user_all_social_spending = user_all_social_matrics.join(spending_profile, (user_all_social_matrics.user == spending_profile.user) & (user_all_social_matrics.period == spending_profile.life_point), how='inner').drop(user_all_social_matrics.user)
display(user_all_social_spending) 

user_tran_1Y,period,recency,frequency,month,friend,friend_of_friend,coeff,pagerank,user,life_point,count_activity,count_food,count_people,count_event,count_travel,count_transportation,count_utility,count_cash,count_illegal
18,7,6,0.03333333333333333,7,19,48,0,1.9987567579915,43,7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0,0,1.0,0,2,2,0,1.799699813336965,946,0,null,null,null,null,null,null,null,null,null
2,0,0,1.0,0,2,1,0,1.3462294727355013,1060,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3,27,0.03333333333333333,3,3,8,0,1.795299629535947,1079,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,12,0.03333333333333333,1,3,6,0,1.638134128758356,1655,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,11,2,0.03333333333333333,11,9,31,0,0.4079666137499459,2391,11,null,null,null,null,null,null,null,null,null
5,2,9,0.03333333333333333,2,4,0,0,1.274906300120366,2492,2,null,null,null,null,null,null,null,null,null
7,12,15,0.03333333333333333,12,5,15,0,1.963047140962693,2847,12,null,null,null,null,null,null,null,null,null
3,9,6,0.03333333333333333,9,4,15,0,2.5647289413563445,3352,9,null,null,null,null,null,null,null,null,null
1,0,0,1.0,0,1,1,0,0.4679993086252958,5269,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# save to storage in case cluster stopped runing
user_all_social_spending.write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/user_all_social_spending.csv")
# display(user_all_social_spending)

In [50]:
# read from storage
user_all_social_spending = spark.read.format('csv').options(header='true', inferSchema='true')\
    .load('dbfs:/FileStore/user_all_social_spending.csv')
user_all_social_spending.createOrReplaceTempView("user_all_social_spending")

In [51]:
#show the columns and select the ones for regression 
user_all_social_spending=user_all_social_spending.fillna(0)

In [52]:
display(user_all_social_spending)

user_tran_1Y,period,recency,frequency,month,friend,friend_of_friend,coeff,pagerank,user,life_point,count_activity,count_food,count_people,count_event,count_travel,count_transportation,count_utility,count_cash,count_illegal
3,8,2,0.06666666666666667,8,3,17,0,0.7417858705101853,863,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,1.0,0,2,3,0,1.301258844533854,1508,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,11,0.03333333333333333,8,3,7,0,1.3064697156888003,1666,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1.0,0,1,6,0,0.7818825780141897,2324,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1.0,0,2,3,0,1.9918457145836768,2351,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,1.0,0,2,6,0,0.6484689750862116,2526,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0,0,1.0,0,6,1,0,3.74559275250642,2719,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,4,23,0.03333333333333333,4,7,9,0,3.74559275250642,2719,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,9,0.03333333333333333,3,2,9,0,1.0452561615539795,2772,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1.0,0,1,3,0,0.5917951279647015,3205,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
# Regress Y on Recency Frequncy variable and user spending profile
mse_vec4 = []
for i in range(13):
    subset = user_all_social_spending.filter(user_all_social_spending['period']==i)
    
    #select independent variables
    assembler = VectorAssembler(inputCols =['friend','friend_of_friend','coeff','pagerank','count_activity','count_food','count_people','count_event','count_travel','count_transportation','count_utility','count_cash','count_illegal'],outputCol ='features')
    output = assembler.transform(subset)
    
    #model data
    finalData = output.select('features','user_tran_1Y')
    
    #regress
    trainData, testData = finalData.randomSplit([0.7, 0.3])
    lrModel_4 = LinearRegression(labelCol ='user_tran_1Y', featuresCol='features')
    lrEstimator_4 = lrModel_4.fit(trainData)
    testResults_4 = lrEstimator_4.evaluate(testData)
    mse = testResults_4.rootMeanSquaredError
    mse_vec4.append(mse)
    
plt.scatter(list(range(13)),mse_vec4)
plt.plot(list(range(13)), mse_vec4)
plt.title('Y on Social Network Metrics and Spending behavior MSE plot')
plt.hlines(np.mean(mse_vec4))
plt.text(1,np.mean(mse_vec4)+0.1,np.mean(mse_vec4))
display(plt.show())

In [54]:
#plot with average reference line
plt.scatter(list(range(13)),mse_vec4)
plt.plot(list(range(13)), mse_vec4)
plt.title('Y on Social Network Metrics and Spending behavior MSE plot')
plt.hlines(np.mean(mse_vec4),0,13)
plt.text(1,np.mean(mse_vec4)+0.02,np.mean(mse_vec4))
display(plt.show())

In [55]:
# regression coefficient of month 12
pd.DataFrame(['friend','friend_of_friend','coeff','pagerank','count_activity','count_food','count_people','count_event','count_travel','count_transportation','count_utility','count_cash','count_illegal'],lrEstimator.coefficients)

,0
0.697240,friend
0.001046,friend_of_friend
1.865867,coeff
0.477672,pagerank
-0.045741,count_activity
0.483750,count_food
0.263516,count_people
-0.077960,count_event
-0.056142,count_travel
0.537350,count_transportation
